In [1]:
import openmm, openmm.app, openmm.unit

In [2]:
import open3SPN2

In [3]:
import sys

In [4]:
import numpy as np

In [5]:
import time

In [12]:
%env X3DNA=/home/ctan/Workspace/x3dna-v2.4

env: X3DNA=/home/ctan/Workspace/x3dna-v2.4


In [14]:
seq = "CC"
dna=open3SPN2.DNA.fromSequence(seq, dna_type='B_curved',output_pdb=f'initial.pdb')
dna.computeTopology(template_from_X3DNA=False)

# Create the system.
# To set periodic boundary conditions (periodicBox=[50,50,50]).
# The periodic box size is in nanometers.

# dna.periodic=True
# s=open3SPN2.System(dna, periodicBox=[50, 50, 50])
dna.periodic=False
s=open3SPN2.System(dna, periodicBox=None)

s.add3SPN2forces(verbose=True)

Bond
Angle
Stacking
Dihedral
BasePair
CrossStacking
Exclusion
Electrostatics


In [45]:
simulation_platform='CUDA' #['Reference', 'CPU', 'OpenCL', 'CUDA']
t300k=300 * openmm.unit.kelvin
s.initializeMD(temperature=t300k,platform_name=simulation_platform)

# setting the ionic strength does not work!!!
# s.forces["Electrostatics"].C = 150

In [46]:
sim=s.simulation

#Set initial positions

sim.context.setPositions(s.coord.getPositions())

In [47]:
energy_unit=openmm.unit.kilocalorie_per_mole
state = sim.context.getState(getEnergy=True)
energy = state.getPotentialEnergy().value_in_unit(energy_unit)
print('TotalEnergy', round(energy,6), energy_unit.get_symbol())

#Obtain detailed energy

energies = {}
for force_name, force in s.forces.items():
    group=force.getForceGroup()
    state = sim.context.getState(getEnergy=True, groups=2**group)
    energies[force_name] =state.getPotentialEnergy().value_in_unit(energy_unit)

for force_name in s.forces.keys():
    print(force_name, round(energies[force_name],6),energy_unit.get_symbol())

TotalEnergy -16.047782 kcal/mol
Bond 0.0 kcal/mol
Angle 9e-06 kcal/mol
Stacking -6.8779 kcal/mol
Dihedral 0.0 kcal/mol
BasePair -8.719931 kcal/mol
CrossStacking -0.464472 kcal/mol
Exclusion 0.0 kcal/mol
Electrostatics 0.014511 kcal/mol
